In [ ]:
#import necessary libraries
import tweepy
import json
import pandas as pd
import glob
import os.path
import time
import nltk
import re 
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score, average_precision_score
from nltk.corpus import stopwords
nltk.download('stopwords') 
nltk.download('wordnet') 

In [ ]:
#Twitter API keysconsumer_key = 'rqPOoqdCS3qpmlpzfuaGVFvZ1'
consumer_secret ='***InsertKeys***'
access_token = '***InsertKeys***'
access_token_secret = '***InsertKeys***'

# OAuth process, using the keys and tokens
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
 
# Creation of the actual interface, using authentication
api = tweepy.API(auth)

In [ ]:
#Stream Listener to get random tweets
class StreamListener(tweepy.StreamListener):
    def on_status(self, status):        
        with open('fetched_tweets.json','a', encoding="utf8") as tf:
            tf.write(status.text)
        
    def on_error(self, status_code):
        if status_code == 420:
            return False
        

#Get tweets in english from US
stream_listener = StreamListener()
stream = tweepy.Stream(auth=api.auth, listener=stream_listener)
stream.filter(languages=["en"], locations = [-122.75,36.8,-121.75,37.8])

In [ ]:
#Real time prediction for fetched tweets done with model trained in RussianTrolls.ipynb
real_tweets = pd.read_csv(r'C:\Users\Chithra Menon\Downloads\RussianTrolls\RealTweets.csv')

#Preprocessing the tweet content in real_tweets

#remove URLs from the tweet content
real_tweets['content'] = real_tweets['content'].str.replace('https?:\/\/.*[\r\n]*', '')

#remove non ASCII characters from tweets
real_tweets['content'] = real_tweets['content'].str.encode('ascii', 'ignore').str.decode('ascii')

#remove words starting with @username as its not relevant to our classification
real_tweets['content'] = real_tweets['content'].str.replace('@(.+?)[\s,.;]', '')

#remove words starting with & as they represent HTML character reference 
real_tweets['content'] = real_tweets['content'].str.replace('&(.+?)[\s,.;]', '')

#remove numerics and special characters except # from the string
real_tweets['content'] = real_tweets['content'].str.replace('[^a-zA-Z#\s]', '')


#Remove stopwords from svm_df
real_tweets['content_without_stopwords'] = real_tweets['content'].str.replace(pat, '')
real_tweets['content_without_stopwords'] = real_tweets['content_without_stopwords'].str.replace(r'\s+', ' ')

#Tokenizing and Lemmatization
real_tweets.fillna('', inplace=True) 

real_tweets['text_lemmatized'] = real_tweets.content_without_stopwords.apply(lemmatize_text)

#convert tweet into lower case
real_tweets['text_lemmatized'] = [entry.lower() for entry in real_tweets['text_lemmatized']]

#Drop unnecessary columns
real_tweets = real_tweets[['text_lemmatized']]

#Word Vectorization with TF-IDF
Real_X_Tfidf = Tfidf_vect.transform(real_tweets['text_lemmatized'])

#Predict outcome
predictions_SVM = SVM.predict(Real_X_Tfidf)

#print outcomes
print(predictions_SVM)